## 특정 컬럼값이 없는 데이터만 삭제하기
  - subset으로 해당 컬럼을 지정해줌

In [12]:
doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
doc = doc.dropna(subset=['Confirmed'])
doc.head()

NameError: name 'PATH' is not defined

## 없는 데이터(NaN)중 특정 컬럼에 대해 특정 값으로 일괄 변경하기
- 별도 사전 데이터를 생성, 없는 데이터를 변경할 컬럼명만 키로 만들고, 변경할 특정 값을 키값으로 넣고, fillna() 함수에 적용해주면 됨

In [ ]:
doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
nan_data = {'Deaths': 0, 'Recovered':0}
doc = doc.fillna(nan_data)
doc.head()

## 컬럼 타입 변경하기
- pandas에서 데이터 타입은 dtype 으로 불리우며, 주요 데이터 타입은 다음과 같음
  - object 는 파이썬의 str 또는 혼용 데이터 타입 (문자열)
  - int64 는 파이썬의 int (정수)
  - float64 는 파이썬의 float (부동소숫점)
  - bool 는 파이썬의 bool (True 또는 False 값을 가지는 boolean)

In [ ]:
- astype({컬럼명: 변경할타입}) : 특정 컬럼의 타입을 변경
  - 변경할 데이터에 없는 데이터(NaN)이 있을 경우, 에러가 날 수 있음

In [ ]:
doc = pd.read_csv(PATH + "01-22-2020.csv", encoding='utf-8-sig')
doc = doc[['Country/Region', 'Confirmed']] # 필요한 컬럼만 선택하기
doc = doc.dropna(subset=['Confirmed'])     # 특정 컬럼에 없는 데이터 삭제하기
doc = doc.astype({'Confirmed': 'int64'})   # 특정 컬럼의 데이터 타입 변경하기
doc.info()

## 데이터프레임에서 중복 행 확인/제거하기
- duplicated() : 중복 행 확인하기
- drop_ducplicates() : 중복 행 삭제중복값
  - 특정 컬럼을 기준으로 중복 행 제거하기
    - subset=특정컬럼
  - 중복된 경우, 처음과 마지막 행 중 어느 행을 남길 것인지 결정하기 
    - 처음: keep='first' (디폴트)
    - 처음: keep='last'

In [ ]:
doc = doc.drop_duplicates(subset='Country_Region', keep='last')
doc

## pandas Dataframe 작성 방법 정리
- pd.DataFrame(data={컬럼이름:컬럼데이터리스트})
  - {컬럼이름:컬럼데이터리스트} 는 사전 데이터 형식
- pd.DataFrame(data=리스트, columns=컬럼이름, index=인덱스데이터)

In [2]:
import numpy as np
import pandas as pd

date_index = pd.date_range('2020-05-01', periods=15)
df = pd.DataFrame(data=range(len(date_index)), columns=['count'], index=date_index)
df

,count
2020-05-01,0
2020-05-02,1
2020-05-03,2
2020-05-04,3
2020-05-05,4
2020-05-06,5
2020-05-07,6
2020-05-08,7
2020-05-09,8
2020-05-10,9


## pandas.Grouper
- pandas groupby 명령에 보다 세부적인 grouping 이 가능토록 하는 명령
- pandas groupby 함수와 함께 쓰여서, 시간별로 데이터를 분류할 수 있는 기능
- 특정 시간별로 grouping 할 수 있음

```
데이터프레임.groupby(pd.Groper(key='그루핑기준이되는 컬럼', freq='세부 기준'))

```

> freq 옵션: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [ ]:
merged_order_month_sum = merged_order_payment_date.groupby(pd.Grouper(key='order_purchase_timestamp', freq='M')).sum() # key 는 기본이 index 임
merged_order_month_sum.head()

## date time

In [ ]:
merged_order_payment_date['order_purchase_timestamp'] = pd.to_datetime(merged_order_payment_date['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
merged_order_payment_date['year'] = merged_order_payment_date['order_purchase_timestamp'].dt.year
merged_order_payment_date['monthday'] = merged_order_payment_date['order_purchase_timestamp'].dt.day
merged_order_payment_date['weekday'] = merged_order_payment_date['order_purchase_timestamp'].dt.weekday
merged_order_payment_date['month'] = merged_order_payment_date['order_purchase_timestamp'].dt.month
merged_order_payment_date['hour'] = merged_order_payment_date['order_purchase_timestamp'].dt.hour
merged_order_payment_date['quarter'] = merged_order_payment_date['order_purchase_timestamp'].dt.quarter
merged_order_payment_date['minute'] = merged_order_payment_date['order_purchase_timestamp'].dt.minute

### 행 임의 이름 순서로 정렬하기
- 우선 해당 이름에 번호를 매겨 놓은 후, sort_values 로 번호를 기반으로 값으로 정렬함

```python
해당데이터프레임Series = pd.Categorical(시리즈데이터, categories=이름순서리스트, ordered=True)
```

In [ ]:
merged_order_payment_hour_weekday['weekday'] = pd.Categorical(merged_order_payment_hour_weekday['weekday'],categories=['Sun','Mon','Tue','Wed','Thu','Fri','Sat'],ordered=True)

## 카테고리별 거래 건수 확인하기
1. 카테고리 이름으로 grouping 한다: products_category_order.groupby('product_category_name_english')
2. grouping 할 때 order는 거래액이 아니라 거래건수로 count 를 한다: .aggregate({'order_id':'count'})
   - .aggregate() 또는 .agg() 를 사용
2. 거래건수로 count 한 값의 컬럼명을 order_count 로 바꾼다: .rename(columns={'order_id':'order_count'})

In [ ]:
import pandas as pd
df = pd.DataFrame({
    "미국": [2.1, 2.2, 2.3],
    "한국": [0.4, 0.5, 0.45],
    "중국": [10, 13, 15]    
})

df.agg({'미국' : 'count', '한국': 'sum'})

In [ ]:
products_category_order = products_category_order.groupby('product_category_name_english').aggregate({'order_id':'count'}).rename(columns={'order_id':'order_count'})